# AV Emotion Training on Colab

This notebook clones your private repo, initializes the audiovisual emotion submodule, preprocesses RAVDESS, and trains a multimodal model. Replace placeholders as needed.

In [ ]:
# --- Setup: clone your private repo and init submodule ---
REPO_URL = "https://github.com/<org-or-user>/schizodot-ai-dot.git"  # If private, use a token URL
!git clone $REPO_URL repo
%cd /content/repo
!git submodule update --init --recursive

# Optional: checkout dev branch
!git checkout dev || echo 'dev branch may already be active'

In [ ]:
# --- Install dependencies from the upstream audiovisual repo ---
!pip install -r external/emotion-av/requirements.txt

In [ ]:
# --- Mount Google Drive to access RAVDESS data ---
from google.colab import drive
drive.mount('/content/drive')

DATA_ON_DRIVE = "/content/drive/MyDrive/data/RAVDESS"  # change to your path
!mkdir -p /content/data
!ln -s "$DATA_ON_DRIVE" /content/data/RAVDESS || echo 'symlink exists'

## Preprocessing
Run the upstream scripts to extract face clips and audio, then build the annotations file. Adjust script names and flags to match the upstream repo if needed.

In [ ]:
# --- Preprocess: examples; adjust to the actual scripts in external/emotion-av ---
!python external/emotion-av/ravdess_preprocessing/extract_faces.py --data_root /content/data/RAVDESS || echo 'adjust script path if needed'
!python external/emotion-av/ravdess_preprocessing/extract_audios.py --data_root /content/data/RAVDESS || echo 'adjust script path if needed'
!python external/emotion-av/ravdess_preprocessing/create_annotations.py --data_root /content/data/RAVDESS --out /content/data/RAVDESS/annotations.txt || echo 'adjust script path if needed'
!head -n 5 /content/data/RAVDESS/annotations.txt || echo 'annotations not ready yet'

## Training
Choose a fusion mode. Example: `ia` stands for intermediate attention. Check the upstream README for options and best settings.

In [ ]:
RESULTS_DIR = "/content/outputs/av_emotion"
!mkdir -p $RESULTS_DIR
!python external/emotion-av/main.py \
  --fusion ia \
  --annotation_path /content/data/RAVDESS/annotations.txt \
  --result_path $RESULTS_DIR || echo 'adjust args to match upstream main.py'

!ls -la $RESULTS_DIR || true

## Save the best checkpoint into your repo
You can keep it on Drive or commit with Git LFS.

In [ ]:
!mkdir -p ai/emotion/weights
!cp $RESULTS_DIR/*best*.pt ai/emotion/weights/av_emotion_best.pt || echo 'best checkpoint not found yet'

## Optional: commit weights using Git LFS
If your repo is private, authenticate with a token. Otherwise skip and store weights on Drive.

In [ ]:
!git lfs install
!git config user.email "you@example.com"
!git config user.name "Your Name"
!git add ai/emotion/weights/av_emotion_best.pt || true
!git commit -m "model: add AV emotion checkpoint" || true
!git push origin dev || echo 'push skipped or not authenticated'